In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.datasets import load_iris
#Iris is available from the sklearn package
iris = load_iris()
X, y = iris.data, iris.target

In [2]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, 
                                                    train_size=0.8,
                                                    test_size=0.2,
                                                    random_state=123,
                                                    stratify=y)

print('All:', np.bincount(y) / float(len(y)) * 100.0)
print('Training:', np.bincount(train_y) / float(len(train_y)) * 100.0)
print('Validation:', np.bincount(val_y) / float(len(val_y)) * 100.0)

All: [33.33333333 33.33333333 33.33333333]
Training: [33.33333333 33.33333333 33.33333333]
Validation: [33.33333333 33.33333333 33.33333333]


In [3]:
batch_size = 60
num_epochs = 500
learning_rate = 0.01
size_hidden= 100

#Calculate some other hyperparameters based on data.  
batch_no = len(train_X) // batch_size  #batches
cols=train_X.shape[1] #Number of columns in input matrix
classes= len(np.unique(train_y))

In [7]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from sklearn.utils import shuffle
from torch.autograd import Variable
running_loss = 0.0
class Net123(torch.nn.Module):
    def __init__(self):
        super(Net123, self).__init__()
        self.fc1 = torch.nn.Linear(4, 100)
        self.fc2 = torch.nn.Linear(100, 3)
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.dropout(x, p=0.1)
        x = F.relu(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

model = Net123()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()
# criterion = torch.nn.MSELoss(size_average=False)

running_loss = 0.0
for epoch in range(num_epochs):
    #Shuffle just mixes up the dataset between epocs
    train_X, train_y = shuffle(train_X, train_y)
    # Mini batch learning
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size
        inputs = Variable(torch.FloatTensor(train_X[start:end]))
        labels = Variable(torch.LongTensor(train_y[start:end]))
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
#         optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    running_loss = 0.0
import mlflow
import mlflow.pytorch
with mlflow.start_run():
    mlflow.log_param("num_epochs", num_epochs)
    # 使用 mlflow  保存模型的时候  pickle_module参数  不填  使用默认值
    # 否则保存的模型  在上架进行测试时候会有错误信息 如该例为“AttributeError: Can't get attribute 'Net123' on <module '__main__'>” 
    # 如果需要使用pickle序列化方式  建议通过将class  Net123 放入Net123.py文件中，log_model之后通过mlflow.log_artifacts('Net123.py','model9/Net123.py')
    '''
    '''
    mlflow.pytorch.log_model(model, artifact_path="model9")

/opt/conda/lib/python3.7/site-packages/torch/serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net123. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
